In [3]:
from heapq import heappush, heappop
import numpy as np

# Goal state of the 8-puzzle
GOAL_STATE = [
    [1, 2, 3],
    [8, 0, 4],
    [7, 6, 5]
]

# Directions for moving the blank tile (up, down, left, right)
MOVES = [(0, -1), (0, 1), (-1, 0), (1, 0)]

# Heuristic function: Manhattan distance
def heuristic(state):
    distance = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0 and abs(state[i][j] - GOAL_STATE[i][j]) != 0:  # Ignore the blank tile
                distance += 1
    return distance

# Check if the current state is the goal state
def is_goal(state):
    return state == GOAL_STATE

# Find the position of the blank tile (0)
def find_blank(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return (i, j)
    return None

# Generate possible moves from the current state
def generate_moves(state):
    blank_row, blank_col = find_blank(state)
    moves = []
    for dr, dc in MOVES:
        new_row, new_col = blank_row + dr, blank_col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_state = [row[:] for row in state]  # Copy the state
            new_state[blank_row][blank_col], new_state[new_row][new_col] = new_state[new_row][new_col], new_state[blank_row][blank_col]
            moves.append(new_state)
    return moves

# A* search algorithm
def a_star_search(start_state):
    for row in start_state:
      print(row)
    print('\nExpanding the Start State\n')
    open_list = []
    heappush(open_list, (heuristic(start_state), 0, start_state, []))  # (f(n), g(n), state, path)
    visited = set()

    while open_list:
        _, g, current_state, path = heappop(open_list)
        if is_goal(current_state):
            return path + [current_state]  # Return the solution path

        if tuple(map(tuple, current_state)) in visited:
            continue
        visited.add(tuple(map(tuple, current_state)))

        print('--------------------------------------------')

        idx = 0
        heuristic_values = []
        for move in generate_moves(current_state):
            if tuple(map(tuple, move)) not in visited:
              for row in move:
                print(row)
              print()
              idx = idx + 1
              print(f"Matrix {idx} : f(n) = {heuristic(move) + g + 1}")
              heuristic_values.append(heuristic(move) + g + 1)
              print()
              heappush(open_list, (g + 1 + heuristic(move), g + 1, move, path + [current_state]))

        print(f"Expanding Matrix: {np.argmin(heuristic_values) + 1} (lowest heuristic value)")
        print()

    return None  # No solution found

# Program execution starts from here
if __name__ == "__main__":
    # Start state
    start_state = [
        [2, 8, 3],
        [1, 6, 4],
        [7, 0, 5]
    ]

    # Solve the puzzle
    solution = a_star_search(start_state)

    if solution:
        print('--------------------------------------------')
        print("Solution found! Steps:")
        for step in solution:
            for row in step:
                print(row)
            print()
    else:
        print("No solution found.")

[2, 8, 3]
[1, 6, 4]
[7, 0, 5]

Expanding the Start State

--------------------------------------------
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]

Matrix 1 : f(n) = 6

[2, 8, 3]
[1, 6, 4]
[7, 5, 0]

Matrix 2 : f(n) = 6

[2, 8, 3]
[1, 0, 4]
[7, 6, 5]

Matrix 3 : f(n) = 4

Expanding Matrix: 3 (lowest heuristic value)

--------------------------------------------
[2, 8, 3]
[0, 1, 4]
[7, 6, 5]

Matrix 1 : f(n) = 5

[2, 8, 3]
[1, 4, 0]
[7, 6, 5]

Matrix 2 : f(n) = 6

[2, 0, 3]
[1, 8, 4]
[7, 6, 5]

Matrix 3 : f(n) = 5

Expanding Matrix: 1 (lowest heuristic value)

--------------------------------------------
[0, 2, 3]
[1, 8, 4]
[7, 6, 5]

Matrix 1 : f(n) = 5

[2, 3, 0]
[1, 8, 4]
[7, 6, 5]

Matrix 2 : f(n) = 7

Expanding Matrix: 1 (lowest heuristic value)

--------------------------------------------
[0, 8, 3]
[2, 1, 4]
[7, 6, 5]

Matrix 1 : f(n) = 6

[2, 8, 3]
[7, 1, 4]
[0, 6, 5]

Matrix 2 : f(n) = 7

Expanding Matrix: 1 (lowest heuristic value)

--------------------------------------------
[1, 2, 3]
[0,

In [4]:
from queue import PriorityQueue
import numpy as np

class PuzzleNode:
    def __init__(self, state, parent=None, move=None, depth=0, cost=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost
        self.mismatch = self.mismatch_count()
        self.total_cost = cost + self.mismatch  # Overall cost: path cost + mismatch count
    
    def __lt__(self, other):
        return self.total_cost < other.total_cost
    
    def mismatch_count(self):
        goal_state = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
        return np.sum(self.state != goal_state) - 1  # Exclude the blank tile
    
    def get_neighbors(self):
        neighbors = []
        x, y = np.where(self.state == 0)
        x, y = int(x), int(y)
        moves = {'Up': (x - 1, y), 'Down': (x + 1, y), 'Left': (x, y - 1), 'Right': (x, y + 1)}
        
        for move, (nx, ny) in moves.items():
            if 0 <= nx < 3 and 0 <= ny < 3:
                new_state = self.state.copy()
                new_state[x, y], new_state[nx, ny] = new_state[nx, ny], new_state[x, y]
                neighbors.append(PuzzleNode(new_state, self, move, self.depth + 1, self.depth + 1))
        
        return neighbors
    
    def path(self):
        node, path = self, []
        while node:
            path.append(node)
            node = node.parent
        return path[::-1]

def solve_puzzle(start_state):
    start_node = PuzzleNode(np.array(start_state))
    goal_state = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
    frontier = PriorityQueue()
    frontier.put(start_node)
    explored = set()
    
    while not frontier.empty():
        node = frontier.get()
        
        if np.array_equal(node.state, goal_state):
            return node.path()
        
        explored.add(tuple(map(tuple, node.state)))
        
        neighbors = node.get_neighbors()
        neighbors.sort(key=lambda n: n.total_cost)  # Sort by overall cost
        
        print(f"Choosing move with lowest total cost at depth {node.depth}:")
        for neighbor in neighbors:
            print(f"Move: {neighbor.move}, Cost: {neighbor.cost}, Mismatch: {neighbor.mismatch}, Total Cost: {neighbor.total_cost}")
            print("State after move:")
            print(neighbor.state)
            print()
        
        if neighbors:
            chosen_neighbor = neighbors[0]
            print("Chosen move with lowest total cost:")
            print(f"Move: {chosen_neighbor.move}, Cost: {chosen_neighbor.cost}, Mismatch: {chosen_neighbor.mismatch}, Total Cost: {chosen_neighbor.total_cost}")
            print("Chosen state:")
            print(chosen_neighbor.state)
            print()
        
        for neighbor in neighbors:
            if tuple(map(tuple, neighbor.state)) not in explored:
                frontier.put(neighbor)
    
    return None

# Example usage:
initial_state = [[1, 2, 3], [0, 4, 6], [7, 5, 8]]
solution = solve_puzzle(initial_state)

if solution:
    for step, node in enumerate(solution):
        print(f"Step {step}: Move {node.move}")
        print(f"Cost: {node.cost}, Mismatch: {max(node.mismatch,0)}, Total Cost: {node.total_cost}")
        print("State:")
        print(node.state)
        print()
else:
    print("No solution found.")


Choosing move with lowest total cost at depth 0:
Move: Right, Cost: 1, Mismatch: 2, Total Cost: 3
State after move:
[[1 2 3]
 [4 0 6]
 [7 5 8]]

Move: Up, Cost: 1, Mismatch: 4, Total Cost: 5
State after move:
[[0 2 3]
 [1 4 6]
 [7 5 8]]

Move: Down, Cost: 1, Mismatch: 4, Total Cost: 5
State after move:
[[1 2 3]
 [7 4 6]
 [0 5 8]]

Chosen move with lowest total cost:
Move: Right, Cost: 1, Mismatch: 2, Total Cost: 3
Chosen state:
[[1 2 3]
 [4 0 6]
 [7 5 8]]

Choosing move with lowest total cost at depth 1:
Move: Down, Cost: 2, Mismatch: 1, Total Cost: 3
State after move:
[[1 2 3]
 [4 5 6]
 [7 0 8]]

Move: Up, Cost: 2, Mismatch: 3, Total Cost: 5
State after move:
[[1 0 3]
 [4 2 6]
 [7 5 8]]

Move: Left, Cost: 2, Mismatch: 3, Total Cost: 5
State after move:
[[1 2 3]
 [0 4 6]
 [7 5 8]]

Move: Right, Cost: 2, Mismatch: 3, Total Cost: 5
State after move:
[[1 2 3]
 [4 6 0]
 [7 5 8]]

Chosen move with lowest total cost:
Move: Down, Cost: 2, Mismatch: 1, Total Cost: 3
Chosen state:
[[1 2 3]
 [4 

C:\Users\mdsho\AppData\Local\Temp\ipykernel_8300\3640313386.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x, y = int(x), int(y)
